In [1]:
# Core
import numpy as np
import pandas as pd
# import scipy
from scipy import interpolate
from scipy.interpolate import interp2d, RectBivariateSpline

from typing import Optional

In [2]:
# MSE imports
import kepler_sieve
from asteroid_element import load_ast_elt, load_data_impl
from astro_utils import anomaly_M2E, anomaly_M2E_impl, anomaly_M2E_make_interp, test_anomaly_M2E

Converged with error 8.88e-16 after 25 iterations.


In [3]:
tau = 2.0 * np.pi

In [4]:
test_anomaly_M2E(10**6)

Converged with error 8.88e-16 after 5 iterations.
Conversion of mean anomaly M to eccentric anomaly E with anomaly_M2E:
Test 1000000 randomly generated input pairs (M, e).
Compare input and recovered M from Keplers Equation M = E - e sin E
Max error = 8.882e-16 = 2^ -50.00
M = 4.870303166855185, e = 0.9216217154070467, E = 4.110592193249946
*** PASS ***


In [5]:
def anomaly_M2E_impl(M: np.ndarray, e: np.ndarray, E0: Optional[np.ndarray] = None) -> np.ndarray:
    """
    Compute the eccentric anomaly E from the mean anomaly M and eccentricity e using and Newton's Method.
    This is the implementation that does not depend on a table of initial guesses
    See: https://en.wikipedia.org/wiki/Eccentric_anomaly
    INPUTS:
        M: The mean anomaly
        e: The eccentricity
        E0: Initial guess
    OUTPUTS:
        E: The Eccentric anomaly

    Kepler's Equation gives us
        M = E - e sin(E)
    which implies that E - e sin(E) - M = 0
    Think of this as a function f(E) = E - e sin(E) - M
    with derivative f'(E) = 1 - e cos(E)
    we solve for f(E) = 0 using Newton's Method
    """

    # Put M in the interval [0, 2*pi)
    M %= tau
    # M = (M + np.pi) % tau - np.pi
    
    # Use the initial guess E0 if provided; otherwise use M
    E = E0 if E0 is not None else M.copy()

    # Maximum number of iterations for Newton-Raphson
    max_iter: int = 40

    # Tolerance for maximum error
    err_tol: np.float64 = 2.0**-49

    # Error for each input vs. iteration number
    err_by_it = np.zeros((max_iter,) + M.shape)

    # Perform at most max_iter iterations of Newton's method; quit early if tolerance achieved    
    for i in range(max_iter):
        # The current function value f(E)
        f = E - e * np.sin(E) - M
        # Save error by iteration
        err_by_it[i] = np.abs(f)
        # Is the max error below the tolerance? If so, quit early
        max_err = np.max(np.abs(f))
        if max_err < err_tol:
            print(f'Converged with error {max_err:5.2e} after {i} iterations.')
            break
        # The derivative f'(E)
        fp = 1.0 - e * np.cos(E)
        # Update E using Newton's method
        E -= f / fp
        # Restore E to range [0, 2 pi) to handle nasty corner cases where very high eccentricity
        # and high M cause a naive iteration to diverge!
        E %= tau
        # E = (E + np.pi) % tau - np.pi

    # Return the converged eccentric anomaly E
    return E, err_by_it

In [7]:
def anomaly_M2E_make_interp(N_M: int = 256, N_e: int = 256) -> interpolate.RectBivariateSpline:
    """Create an interpolation function for inital guesses to compute E from M and e."""
    # Rows of M: N_M evenly spaced angles in [0, 2 pi), e.g. [0, 45, 90, ... 315] degrees for N_M = 8
    M_ = np.arange(N_M) * (tau / N_M)
    # Repeat the rows of M N_e times
    M = np.tile(M_, (N_e, 1))

    # Columns of e: N_e evenly spaced eccentricities in [0, 1), e.g. [0, 0.25, 0.5, 0.75] for N_e = 4
    # e_ = np.arange(N_e) / N_e

    # Columns of e: N_e evenly alog spaced eccentricities in [0, 1 - 2^-48]
    log_e_ = np.linspace(0.0, -48.0*np.log(2.0), N_e)
    e_ = 1.0 - np.exp(log_e_)
    # Repeat the columns of e N_m times
    e = np.tile(e_, (N_M, 1)).T

    # Compute the eccentric anomaly E using the implementation version of anomaly_M2E
    # Take transpose so shape is N_M x N_e to match API of interpolate.RectBivariateSpline
    E0, _ = anomaly_M2E_impl(M, e)
    E0 = E0.T

    # Create a 2D interpolator
    interp = interpolate.RectBivariateSpline(x=M_, y=e_, z=E0, kx=3, ky=1)
    
    return interp

In [8]:
# interp_M2E = anomaly_M2E_make_interp(2**16, 2**8)

In [9]:
# N_M = 2**12
N_M = 2**12
N_e = 2**8

In [10]:
# Rows of M: N_M evenly spaced angles in [0, 2 pi), e.g. [0, 45, 90, ... 315] degrees for N_M = 8
# M_ = np.arange(N_M) * (tau / N_M)
M_ = tau * np.linspace(0.0, 1.0, N_M+1)
# Repeat the rows of M N_e times
M = np.tile(M_, (N_e, 1))

# Columns of e: N_e evenly alog spaced eccentricities in [0, 1 - 2^-48]
log_e_ = np.linspace(0.0, -48.0*np.log(2.0), N_e)
e_ = 1.0 - np.exp(log_e_)
# Repeat the columns of e N_m times
e = np.tile(e_, (N_M+1, 1)).T

# Compute the eccentric anomaly E using the implementation version of anomaly_M2E
# Take transpose so shape is N_M x N_e to match API of interpolate.RectBivariateSpline
E_, _ = anomaly_M2E_impl(M, e)
# Overwrite the last column, for M= 360 degrees, with E= 360 degrees
E_[:, -1] = tau

E_ = E_.T

Converged with error 8.88e-16 after 24 iterations.


In [43]:
# Create a 2D interpolator
interp_M2E = interpolate.RectBivariateSpline(x=M_, y=e_, z=E_, kx=5, ky=3)

In [44]:
M = np.array([6.283010303931326])
e = np.array([0.9937009835495966])
E = np.array([6.255934715346866])

In [45]:
E0 = interp_M2E.ev(M, e)
E0

array([6.25713799])

In [46]:
E - E0

array([-0.00120327])

In [47]:
anomaly_M2E_impl(M, e, E0)[0]

Converged with error 0.00e+00 after 3 iterations.


array([6.25593472])

In [48]:
anomaly_M2E(M, e)

Converged with error 0.00e+00 after 4 iterations.


array([6.25593472])

In [49]:
from asteroid_element import load_ast_elt

In [ ]:
df = load_ast_elt()

In [ ]:
M = df.M.values
e = df.e.values

In [ ]:
E0 = interp_M2E.ev(M, e)

In [ ]:
E_impl = anomaly_M2E_impl(M=M, e=e)

In [ ]:
E = anomaly_M2E(M=M, e=e)

In [ ]:
M1 = E0 - e * np.sin(E0)

In [ ]:
M2 = E - e * np.sin(E)

In [ ]:
err1 = np.abs(M1 - M)
err2 = np.abs(M2 - M)

In [ ]:
np.max(err1)

In [ ]:
np.max(err2)

In [ ]:
err = np.abs(E - E_impl)

In [ ]:
np.max(err)

In [ ]:
E[0:5]

In [ ]:
E_impl[0:5]

In [ ]:
idx = np.argmax(err)

In [ ]:
M_ = M[idx]
M_

In [ ]:
e_ = e[idx]
e_

In [ ]:
e_ = 0.95

In [ ]:
E1 = anomaly_M2E_impl(M_, e_)
E1

In [ ]:
E2 = interp_M2E.ev(M_, e_)
E2

In [ ]:
np.abs(E1-E2)

In [ ]:
E0[idx]

In [ ]:
E[idx]

In [ ]:
interp_M2E(x=0.2, y=0.1)

In [ ]:
interp_M2E(x=[0.1, 0.2], y=[0.1, 0.2, 0.3])

In [ ]:
M.shape

In [ ]:
e.shape

In [ ]:
E0 = interp_M2E.ev([0.1, 0.2], [0.0, 0.0])

In [ ]:
E0

In [ ]:
E0 = interp_M2E.ev(M, e)

In [ ]:
E0

In [ ]:
e[0]

In [ ]:
E_impl[0]

In [ ]:
E0[0]

In [ ]:
interp_M2E(x=0.2, y=0.1)

In [ ]:
test_anomaly_M2E()

In [ ]:
M = np.array([4.870303166855185])
e = np.array([0.9216217154070467])
E = np.array([4.110592193249945])

In [ ]:
anomaly_M2E_impl(M, e)

In [ ]:
E - e * np.sin(E) - M

In [ ]:
np.log(np.abs(E - e * np.sin(E) - M))

In [ ]:
# Put M in the interval [0, 2*pi)
M %= tau
M

In [ ]:
# Use the initial guess E0 if provided; otherwise use M
E = M.copy()
E

In [ ]:
# Maximum number of iterations for Newton-Raphson
max_iter: int = 30 

In [ ]:
# Tolerance for maximum error
err_tol: np.float64 = 2.0**-50

In [ ]:
# Perform at most max_iter iterations of Newton's method; quit early if tolerance achieved
for i in range(max_iter):
    # The current function value f(E)
    f = E - e * np.sin(E) - M
    # Is the max error below the tolerance? If so, quit early
    max_err = np.max(np.abs(f))
    if max_err < err_tol:
        # print(f'Converged with error {max_err:5.2e} after {i+1} iterations.')
        break
    # The derivative f'(E)
    fp = 1.0 - e * np.cos(E)
    # Update E using Newton's method
    E -= f / fp

In [ ]:
i

In [ ]:
f

In [ ]:
E

In [ ]:
e

In [ ]:
M

In [ ]:
E - e * np.sin(E) - M

In [ ]:
E

In [ ]:
f